In [ ]:
# Code By adityatb at https://github.com/adityatb/noise-reduction-using-rnn
# LSTM method test.
# Maintain by ShYy, 2018.

import scipy
import scipy.signal as signal
import numpy as np
import os, random, sys
import scipy.io.wavfile as wav
import math
import pandas as pd

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [ ]:
%tensorflow_version 1.8
import tensorflow as tf

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.8`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [ ]:

file_repository = []

for root, _, files in os.walk(traindata):
    files = sorted(files)

    #files를 읽어와 data와 samplingrate를 각각 리스트에 저장
    for f in files:
        if f.endswith(".wav"):
            srate, data = wav.read(os.path.join(root, f))
            file_repository.append(data)
            print(f)

#training data를 pickle로 변환
#dataset = pd.DataFrame(columns = ['Output'])
#dataset.Output = file_repository
#dataset.to_pickle("/content/gdrive/My Drive/Data/dataset_output.pkl") # pkl로 저장


NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/My Drive/Data/

/content/gdrive/My Drive/Data


In [ ]:
os.getcwd()

'/content/gdrive/My Drive/Data'

In [ ]:
traindata = os.getcwd() + "/Testing/ModelOutput3/"

In [ ]:
#filename을 입력받아 앞에서부터 len(filename)-11 까지만 남긴 뒤 _voice 추가
def formatFilename(filename):
    return filename[:len(filename) - 11] + "_voice.wav"


#Strip away the _xnoise.wav part of the filename, and append _voice.wav to obtain clean voice counterpart
def create_final_sequence(sequence, max_length):
    a, b = sequence.shape
    extra_len = max_length - b
    null_mat = np.zeros((len(sequence), extra_len), dtype=np.float32)
    sequence = np.concatenate((sequence, null_mat), axis=1)
    return sequence


def sequentialized_spectrum(batch):
    # Get maximum length of batch
    t = []
    t_vec = []
    Sxx_Vec = []
    for each in batch:
        _, t, Sxx_Vec_Temp = signal.stft(each, fs=rate_repository[0], nperseg=stft_size, return_onesided = False) #_ 는 frequency, t는 segment time, STFT of x
        t_vec.append(t)   #segment time 저장
        Sxx_Vec.append(Sxx_Vec_Temp)    #여기에 ft된 data가 저장되는듯
    maximum_length = findMaxlen(t_vec)  #최대의 시간값저장?

    max_run_total = int(math.ceil(float(maximum_length) / sequence_length))
    final_data = np.zeros([len(batch), max_run_total, stft_size, sequence_length])
    true_time = np.zeros([len(batch), max_run_total])

    # Read in a file and compute spectrum
    # for batch_idx, each_set in enumerate(batch):
    for batch_idx, Sxx in enumerate(Sxx_Vec):
        # f, t, Sxx = signal.stft(each_set, fs=rate_repository[0], nperseg=stft_size, return_onesided = False)

        # Magnitude and Phase Spectra
        Mag = Sxx.real
        t = t_vec[batch_idx]
        # Phase = Sxx.imag

        # Break up the spectrum in sequence_length sized data
        run_full_steps = float(len(t)) / sequence_length
        run_total = int(math.ceil(run_full_steps))

        # Run a loop long enough to break up all the data in the file into chunks of sequence_size
        for step in range(run_total):

            begin_point = step * sequence_length
            end_point = begin_point + sequence_length

            m, n = Mag[:, begin_point:end_point].shape

            # Store each chunk sequentially in a new array, accounting for zero padding when close to the end of the file
            if n == sequence_length:
                final_data[batch_idx, step, :, :] = np.copy(Mag[:, begin_point:end_point])
                true_time[batch_idx, step] = n
            else:
                final_data[batch_idx, step, :, :] = np.copy(create_final_sequence(Mag[:, begin_point:end_point], sequence_length))
                true_time[batch_idx, step] = n

    final_data = np.transpose(final_data, (0, 1, 3, 2))

    return final_data, true_time, maximum_length

#입력받은 datafile 내의 최대 len을 return
def findMaxlen(data_vec):
    max_ = 0
    for each in data_vec:
        if len(each) > max_:
            max_ = len(each)
    return max_



In [ ]:
# ----------------- Begin Vars --------------------- #

# Training data directories
traindata = os.getcwd() + "/Training/NoiseAdded2/"
voicedata = os.getcwd() + "/Training/HumanVoices/"
checkpoints = os.getcwd() + "/TF_Checkpoints/"

# NormConstant
norm_factor = (1 / 32768.0)

# Spectrogram Parameters
stft_size = 1024

# RNN Specs
sequence_length = 100
batch_size = 1200
learning_rate = 0.001
epochs = 250
# number_of_layers = 3

# Tensorflow vars + Graph and LSTM Params
input_data = tf.placeholder(tf.float32, [None, sequence_length, stft_size]) #input
clean_data = tf.placeholder(tf.float32, [None, sequence_length, stft_size]) #label
sequence_length_tensor = tf.placeholder(tf.int32, (None)) 

# Temp_data_variables
no_of_files = 0
temp_list = []
final_data = []
sequence_length_id = 0

# Repositories
file_repository = []
rate_repository = []
clean_repository = []

# Selected vectors
files_vec = []
clean_files_fin_vec = []
clean_files_vec = []

# Graph
lstm_cell = tf.contrib.rnn.BasicLSTMCell(stft_size, forget_bias = 1.0, state_is_tuple = True)
#lstm Cell 생성

# stacked_lstm = tf.contrib.rnn.MultiRNNCell([[lstm_cell] for i in number_of_layers])
#여기선 빠른 학습을 위해 아마 층을 안쌓은것 같음

init_state = lstm_cell.zero_state(batch_size, tf.float32)
#초기상태를 zero로 초기화

rnn_outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, input_data, sequence_length=sequence_length_tensor, initial_state=init_state, time_major=False)
#TensorFlow dynamic_rnn을 사용하면 모델의 결과와 마지막 상태값을 반환하는데, 이것들이 학습하는 동안 배치 사이에서 전달되어야 한다.

mse_loss = tf.losses.mean_squared_error(rnn_outputs, clean_data)
#rnn_output을 이용 loss함수는 mse를 사용한다.

# train_optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(mse_loss)
# train_optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(mse_loss)
# train_optimizer = tf.train.AdagradDAOptimizer(learning_rate).minimize(mse_loss)
train_optimizer = tf.train.AdamOptimizer(learning_rate).minimize(mse_loss)
#optimizer로는 Adam을 사용

saver = tf.train.Saver()
#tf.train.Saver()함수를 이용해 만들어놓은 cell 과 layer, optimizer, lossfunction 을 저장한다.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
# ------------------- Read all data to memory creating a repository of mixture and clean files --------------------- #

os.chdir(traindata)
# for file_iter in range(traindata):

# Buffer training data to memory for faster execution:
for root, _, files in os.walk(traindata):
    files = sorted(files)
    no_of_files = len(files)
    

    #files는 NoiseAdded data들을 의미하며 배치사이즈가 training data size보다 크면 말이 안되니까 여기서 에러처리
    if batch_size > no_of_files:
        sys.exit("Error: batch_size cannot be more than number of files in the training directory")

    #files를 읽어와 data와 samplingrate를 각각 리스트에 저장
    for f in files:
        if f.endswith(".wav"):
            temp_list.append(f)
            srate, data = wav.read(os.path.join(root, f))
            file_repository.append(data)
            rate_repository.append(srate)

#training data를 pickle로 변환
#dataset = pd.DataFrame(columns = ['Audio'])
#dataset.Audio = file_repository
#dataset.to_pickle("/content/gdrive/My Drive/Data/dataset.pkl") # pkl로 저장

In [ ]:
#df = pd.read_pickle("/content/gdrive/My Drive/Data/dataset.pkl") #pkl 불러오기

In [ ]:
#df # python에서 처리하는 방법

In [ ]:
# Generate a vector of file names that are clean files
#map은 (함수,인자) 를 입력으로 받고 들어온 모든 인자에 대해 함수를 실행시켜 리스트형태로 반환시켜주는 함수이다 고로 들어온 temp_list 인자들에 대해 formatFilename이 실행된다
#간단히 말하면 기존의 이름을 바꾸는 역할을 한다
clean_files_vec = list(map(formatFilename, temp_list))
# clean_files_vec = list(map(None, *clean_files_vec))

# Find clean files that correspond to data in file_repository and buffer clean voice data to memory
#clean한 목소리 데이터들을 clean_repasitory에 저장
for root, _, files in os.walk(voicedata):
    files = sorted(files)
    for each in files:
        if each.endswith(".wav"):
            for name in clean_files_vec:
                if each == name:
                    srate2, data2 = wav.read(os.path.join(root, name))
                    clean_repository.append(data2)

#training data를 pickle로 변환
#dataset2 = pd.DataFrame(columns = ['Voice'])
#dataset2.Voice = clean_repository
#dataset2.to_pickle("/content/gdrive/My Drive/Data/dataset_voice.pkl") # pkl로 저장

"""
#최종적으로
file_repository
rate_repository
clean_repository
가 생성됨
"""

'\n#최종적으로\nfile_repository\nrate_repository\nclean_repository\n가 생성됨\n'

In [ ]:
# files_vec = []
run_epochs = int((no_of_files / batch_size) * epochs)
print(run_epochs)
#no_of_files는 training 파일의 개수이니 3000
#batch_size는 10으로 설정됨
#epochs 는 250 이므로
#run_epochs 의 값은 300 * 250 = 75000
1 * 250

1041


250

In [ ]:
# ------------------- Step 1: Prepare data in batches and perform STFTs --------------------- #
# Initialize TF Graph
init_op = tf.global_variables_initializer()  # initialize_all_variables()
gpu_options = tf.GPUOptions(allow_growth = True)            #탄력적으로 GPU memory 를 할당하는 코드

sess = tf.Session(config = tf.ConfigProto(gpu_options = gpu_options))
sess.run(init_op) #deep learning 실행!

In [22]:
globalBatchLossSum = 0          # Sum of all batch losses
globalStepsSum = 0          # Sum of all steps
lastCumulativeLossAvg = 100           # Last Cumulative Loss Avg.

#이 for문은 무슨 작업이지??
#run_epochs((no_of_files / batch_size) * epochs) 만큼 실행
for idx in range(int(1)):

    files_vec = []
    #clean_files_vec = []
    clean_files_fin_vec = []

    #Select batch_size no. of random number of files from file_repository and the corresponding clean files
    for file_iter in range(batch_size):
        i = random.randint(0, len(file_repository) - 1)                     #효율적인 학습을위해 file_vec에 랜덤으로 저장 
        files_vec.append(file_repository[i] * norm_factor)                  #file_vec 에있는   data와 clearn_files에있는 data 가 index가 일치되게 정렬되어있는가?
        clean_files_fin_vec.append(clean_repository[i] * norm_factor)
    stft_batch, sequence_length_id, maximum_length = sequentialized_spectrum(files_vec)
    clean_voice_batch, sequence_length_id_clean, maximum_length_clean = sequentialized_spectrum(clean_files_fin_vec)

    # ------------------- Step 2: Feed Data to Placeholders, and then, Initialise, Train and Save the Graph  --------------------- #

    max_time_steps = stft_batch.shape[1]
    batchLossSum = 0            # Sum of batch losses in one index.

    #만들어놓은 placeholder에 data를 집어 넣는다.
    #왜 이걸 max_time_steps 만큼?
    for time_seq in range(max_time_steps):
        feed_dict = {
            input_data: stft_batch[:, time_seq, :, :],
            clean_data: clean_voice_batch[:, time_seq, :, :],
            sequence_length_tensor: sequence_length_id[:, time_seq]
        }
        _, loss_value, final_state_value, rnn_outputs_val = sess.run([train_optimizer, mse_loss, final_state, rnn_outputs], feed_dict=feed_dict)

        # print("Index " + str(idx + 1) + " in " + str(run_epochs))
        # print("\tOutput Min:\t" + str(np.min(rnn_outputs_val)))
        # print("\tClean Min:\t" + str(np.min(clean_voice_batch[:, time_seq, :, :])))
        # print("\tOutput Max:\t" + str(np.max(rnn_outputs_val)))
        # print("\tClean Max:\t" + str(np.max(clean_voice_batch[:, time_seq, :, :])))
        # print("\tBatch Loss:\t" + str(loss_value * 32768))          # Multiplied 32768 to show the batch losses obviously.
        batchLossSum = batchLossSum + loss_value

    print("\t\tIndex " + str(idx + 1) + " Batch Loss Avg:\t" + str(batchLossSum / max_time_steps / norm_factor) + "\n")

    globalBatchLossSum = globalBatchLossSum + batchLossSum
    globalStepsSum = globalStepsSum + max_time_steps

    #if (int((idx + 1) % no_of_files) == 0):
    if (int((idx + 1) % 500) == 0):
        # All batch losses sum divide global steps to get Avg
        cumulativLossAvg = globalBatchLossSum / globalStepsSum
        print("\n\t\tCumulative epochs loss Avg in latest " + str(idx + 1) + " indexes:\t" + str(cumulativLossAvg / norm_factor))
        if(cumulativLossAvg <= lastCumulativeLossAvg):
            lastCumulativeLossAvg = cumulativLossAvg            # If cumulative loss avg is smaller or equal to last avg, stay learning rate
        else:
            learning_rate = learning_rate / 5           # If cumulative loss avg is bigger than last avg, than change learning rate to 1/5
            lastCumulativeLossAvg = cumulativLossAvg
            print("\n\t\tLearning Rate changed to: " + str(learning_rate))
        globalBatchLossSum = 0          # Initialize to 0, for next indexes batch loss calculation
        globalStepsSum = 0

        os.chdir(checkpoints)
        saver.save(sess, './ssep_model_increaseBatchSize.ckpt', global_step=idx)
        print("\t\tSaved checkpoint\n")
        os.chdir(traindata)

os.chdir(checkpoints)
saver.save(sess, './FINAL.ckpt')
print("Saved FINAL")
sess.close()

RuntimeError: ignored

In [19]:
max_time_steps = stft_batch.shape[1]

In [20]:
max_time_steps

4

In [18]:
stft_batch.shape

(1200, 4, 100, 1024)

In [14]:
globalBatchLossSum = 0          # Sum of all batch losses
globalStepsSum = 0          # Sum of all steps
lastCumulativeLossAvg = 100           # Last Cumulative Loss Avg.

#이 for문은 무슨 작업이지??
#run_epochs((no_of_files / batch_size) * epochs) 만큼 실행
for idx in range(int(run_epochs)):

    files_vec = []
    #clean_files_vec = []
    clean_files_fin_vec = []

    #Select batch_size no. of random number of files from file_repository and the corresponding clean files
    for file_iter in range(batch_size):
        i = random.randint(0, len(file_repository) - 1)                     #효율적인 학습을위해 file_vec에 랜덤으로 저장 
        files_vec.append(file_repository[i] * norm_factor)                  #file_vec 에있는   data와 clearn_files에있는 data 가 index가 일치되게 정렬되어있는가?
        clean_files_fin_vec.append(clean_repository[i] * norm_factor)
    stft_batch, sequence_length_id, maximum_length = sequentialized_spectrum(files_vec)
    clean_voice_batch, sequence_length_id_clean, maximum_length_clean = sequentialized_spectrum(clean_files_fin_vec)

    # ------------------- Step 2: Feed Data to Placeholders, and then, Initialise, Train and Save the Graph  --------------------- #

    max_time_steps = stft_batch.shape[1]
    batchLossSum = 0            # Sum of batch losses in one index.

    #만들어놓은 placeholder에 data를 집어 넣는다.
    #왜 이걸 max_time_steps 만큼?
    for time_seq in range(max_time_steps):
        feed_dict = {
            input_data: stft_batch[:, time_seq, :, :],
            clean_data: clean_voice_batch[:, time_seq, :, :],
            sequence_length_tensor: sequence_length_id[:, time_seq]
        }
        _, loss_value, final_state_value, rnn_outputs_val = sess.run([train_optimizer, mse_loss, final_state, rnn_outputs], feed_dict=feed_dict)

        # print("Index " + str(idx + 1) + " in " + str(run_epochs))
        # print("\tOutput Min:\t" + str(np.min(rnn_outputs_val)))
        # print("\tClean Min:\t" + str(np.min(clean_voice_batch[:, time_seq, :, :])))
        # print("\tOutput Max:\t" + str(np.max(rnn_outputs_val)))
        # print("\tClean Max:\t" + str(np.max(clean_voice_batch[:, time_seq, :, :])))
        # print("\tBatch Loss:\t" + str(loss_value * 32768))          # Multiplied 32768 to show the batch losses obviously.
        batchLossSum = batchLossSum + loss_value

    print("\t\tIndex " + str(idx + 1) + " Batch Loss Avg:\t" + str(batchLossSum / max_time_steps / norm_factor) + "\n")

    globalBatchLossSum = globalBatchLossSum + batchLossSum
    globalStepsSum = globalStepsSum + max_time_steps

    #if (int((idx + 1) % no_of_files) == 0):
    if (int((idx + 1) % 500) == 0):
        # All batch losses sum divide global steps to get Avg
        cumulativLossAvg = globalBatchLossSum / globalStepsSum
        print("\n\t\tCumulative epochs loss Avg in latest " + str(idx + 1) + " indexes:\t" + str(cumulativLossAvg / norm_factor))
        if(cumulativLossAvg <= lastCumulativeLossAvg):
            lastCumulativeLossAvg = cumulativLossAvg            # If cumulative loss avg is smaller or equal to last avg, stay learning rate
        else:
            learning_rate = learning_rate / 5           # If cumulative loss avg is bigger than last avg, than change learning rate to 1/5
            lastCumulativeLossAvg = cumulativLossAvg
            print("\n\t\tLearning Rate changed to: " + str(learning_rate))
        globalBatchLossSum = 0          # Initialize to 0, for next indexes batch loss calculation
        globalStepsSum = 0

        os.chdir(checkpoints)
        saver.save(sess, './ssep_model_increaseBatchSize.ckpt', global_step=idx)
        print("\t\tSaved checkpoint\n")
        os.chdir(traindata)

os.chdir(checkpoints)
saver.save(sess, './FINAL.ckpt')
print("Saved FINAL")
sess.close()

KeyboardInterrupt: ignored